In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Dataset
TMNIST: A database of Typeface based digits
This dataset is inspired by the MNIST database for handwritten digits. It consists of images representing digits from 0-9 produced using 2,990 google fonts files. 
The dataset consists of a single file:
TMNIST_Data.csv
This file consists of 29,900 examples with labels and font names. Each row contains 786 elements: the first element represents the font name (ex-Chivo-Italic, Sen-Bold), the second element represents the label (a number from 0-9) and the remaining 784 elements represent the grayscale pixel values (from 0-255) for the 28x28 pixel image.

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the TMNIST CSV File

In [ ]:
data = pd.read_csv("../input/tmnist-typeface-mnist/TMNIST_Data.csv")
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
print(data['names'].unique())

In [ ]:
data['names'].value_counts()

In [ ]:
len(data)

# Split Sample set and Label

In [ ]:
X = data.drop({'labels','names'},axis=1)
y = data[['labels']]

In [ ]:
# Sample Set
X.head()

In [ ]:
# Labels
y.head()

# Sample Images

In [ ]:
X_images = X.values.reshape(-1,28,28)
fig,axs = plt.subplots(3,3,figsize=(9,9))
for i in range(9):
    r=i//3
    c=i%3
    axs[r][c].set_xticks([])
    axs[r][c].set_yticks([])
    axs[r][c].imshow(X_images[i])
plt.show()

# Train Test Dataset Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
y_train = y_train.values.reshape((-1,))
y_test = y_test.values.reshape((-1,))

In [ ]:
print(f"1. Length of X_train: {X_train.shape} \n2. Length of X_test: {X_test.shape}")

# Logistic Classification

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=47, solver='sag')

In [ ]:
# Train the model

model.fit(X_train, y_train)

In [ ]:
# Predict the Test Dataset

prediction = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

print(f"Accuracy for Train: {accuracy_score(y_train, model.predict(X_train))}")
print(f"Accuracy for Test: {accuracy_score(y_test, prediction)}")

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, prediction)

# SVC Classifier

In [ ]:
# Hyperparameter tuning for SVC using BayesSearch

from sklearn.svm import SVC
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

svc_model = BayesSearchCV(SVC(),
             {
                 'C': Real(1e-6, 1e+6, prior='log-uniform'),
                 'gamma': Real(1e-6, 1e+1, prior='log-uniform'),
                 'degree': Integer(1,8),
                 'kernel': Categorical(['linear', 'poly', 'rbf', 'sigmoid']),
                 'decision_function_shape':Categorical(['ovo'])
             },
             n_iter=60,
             random_state=133)


In [ ]:
# Traing the model
svc_model.fit(X_train, y_train)

In [ ]:
# Predict the Test Dataset
svc_prediction = model.predict(X_test)

In [ ]:
print(f"SVC Accuracy for Train: {accuracy_score(y_train, model.predict(X_train))}")
print(f"SVC Accuracy for Test: {accuracy_score(y_test, svc_prediction)}")

In [ ]:
confusion_matrix(y_test, svc_prediction)